In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# creating a def for monthly calculation 
def monthly(x, value):
    if x in [409,419, 619, 649]:
        return (value * 30)/ 365
    elif x in [169, 219, 239, 249, 189, 269, 279, 199, 289, 299, 309, 319, 329]:
        return (value * 30)/ 7
    else:
        return value

In [3]:
def split_string(s):
    part0 = s[:13]   # first 2 bytes
    part1 = s[13:14]  # first 2 bytes
    part2 = s[14:16]  # next 2 bytes
    part3 = s[16:]   # remaining bytes
    return pd.Series([part0,part1, part2, part3]) 

In [4]:
col_specification=[(0,38),(38,39),(39,41),(41,46),(46,49),(49,59),(59,74)]
col_names=['Common-ID','Questionnaire No.','Level','Section','Item Code','Value (in Rs)','Multiplier']#,'Marital Status (code)','Highest educational level attained (code)','Total year of education completed','Whether used internet from any location during last 30 days']
lvl_14= pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_14.txt', header=None, colspecs=col_specification, names = col_names)
lvl_14.head(8)

In [ ]:
level_14 = lvl_14.drop(['Level','Section'],axis=1)
level_14

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,139,90,8890.0
1,HCES2022655621010121713017 101111 310,F,139,90,8890.0
2,HCES2022655621010121713017 101111 311,F,139,25,8890.0
3,HCES2022655621010121713017 101111 313,F,139,45,8890.0
4,HCES2022655371010122023014 201111 202,F,139,75,140500.0
...,...,...,...,...,...
8075523,HCES20223492023737102120110201201 313,D,579,3450,938.0
8075524,HCES20223492023737102120110201201 313,D,599,1700,938.0
8075525,HCES20223492023737102120110201201 313,D,629,1280,938.0
8075526,HCES20223492023737102120110201201 313,D,639,5350,938.0


In [ ]:
# for checking is that value of milk prodect in a single housee hold
temp = level_14[level_14['Item Code'].isin([169])]
temp

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier
12875,HCES2022653651030311613011 103123 2301,F,169,630,150165.0
12885,HCES2022653651030311613011 103123 2302,F,169,935,150165.0
12896,HCES2022653651030311613011 103123 2303,F,169,560,150165.0
12904,HCES2022653651030311613011 103123 2305,F,169,1380,150165.0
12912,HCES2022653651030311613011 103123 2306,F,169,518,150165.0
...,...,...,...,...,...
2669949,HCES20223492023737102120110201201 309,F,169,700,938.0
2669960,HCES20223492023737102120110201201 310,F,169,525,938.0
2669970,HCES20223492023737102120110201201 311,F,169,700,938.0
2669980,HCES20223492023737102120110201201 312,F,169,525,938.0


In [ ]:
level_14['Monthly Value'] = level_14.apply(lambda row: monthly(row['Item Code'], row['Value (in Rs)']), axis=1)
level_14

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,8890.0,90.000000
1,HCES2022655621010121713017 101111 310,F,139,90,8890.0,90.000000
2,HCES2022655621010121713017 101111 311,F,139,25,8890.0,25.000000
3,HCES2022655621010121713017 101111 313,F,139,45,8890.0,45.000000
4,HCES2022655371010122023014 201111 202,F,139,75,140500.0,75.000000
...,...,...,...,...,...,...
8075523,HCES20223492023737102120110201201 313,D,579,3450,938.0,3450.000000
8075524,HCES20223492023737102120110201201 313,D,599,1700,938.0,1700.000000
8075525,HCES20223492023737102120110201201 313,D,629,1280,938.0,1280.000000
8075526,HCES20223492023737102120110201201 313,D,639,5350,938.0,5350.000000


In [ ]:
level_14['Multiplier'] = level_14['Multiplier']/100
level_14

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,88.90,90.000000
1,HCES2022655621010121713017 101111 310,F,139,90,88.90,90.000000
2,HCES2022655621010121713017 101111 311,F,139,25,88.90,25.000000
3,HCES2022655621010121713017 101111 313,F,139,45,88.90,45.000000
4,HCES2022655371010122023014 201111 202,F,139,75,1405.00,75.000000
...,...,...,...,...,...,...
8075523,HCES20223492023737102120110201201 313,D,579,3450,9.38,3450.000000
8075524,HCES20223492023737102120110201201 313,D,599,1700,9.38,1700.000000
8075525,HCES20223492023737102120110201201 313,D,629,1280,9.38,1280.000000
8075526,HCES20223492023737102120110201201 313,D,639,5350,9.38,5350.000000


In [ ]:
# for checking is that value of milk prodect in a single housee hold per month
temp1 = level_14[level_14['Item Code'].isin([169])]
temp1

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
12875,HCES2022653651030311613011 103123 2301,F,169,630,1501.65,2700.000000
12885,HCES2022653651030311613011 103123 2302,F,169,935,1501.65,4007.142857
12896,HCES2022653651030311613011 103123 2303,F,169,560,1501.65,2400.000000
12904,HCES2022653651030311613011 103123 2305,F,169,1380,1501.65,5914.285714
12912,HCES2022653651030311613011 103123 2306,F,169,518,1501.65,2220.000000
...,...,...,...,...,...,...
2669949,HCES20223492023737102120110201201 309,F,169,700,9.38,3000.000000
2669960,HCES20223492023737102120110201201 310,F,169,525,9.38,2250.000000
2669970,HCES20223492023737102120110201201 311,F,169,700,9.38,3000.000000
2669980,HCES20223492023737102120110201201 312,F,169,525,9.38,2250.000000


In [ ]:
# avg value of monthly 
temp1['Monthly Value'].mean()

np.float64(1660.4350897524832)

In [ ]:
# description value of monthly 
temp1['Monthly Value'].describe()

count    247522.000000
mean       1660.435090
std        1991.656250
min          21.428571
25%         642.857143
50%        1200.000000
75%        2142.857143
max      563571.428571
Name: Monthly Value, dtype: float64

# Calculating total expanditure on the base of monthly value
mult_applied = level_14.copy()
mult_applied['Total Expenditure'] = mult_applied['Monthly Value']*mult_applied['Multiplier']
mult_applied

In [ ]:
col_specification=[(0,38),(38,39),(39,41),(41,43),(43,44),(44,45),(45,48),(48,49),(49,51),(51,53),(53,54),(54,56),(56,57),(57,59),(59,61),(61,63),(63,65),(65,67),(67,68),(68,69),(69,84)]
col_names=['Common-ID','Questionnaire No.','Level','Person Srl No.','Relation to head (code)','Gender','Age(in years)','Marital Status (code)','Highest educational level attained (code)','Total year of education completed','Whether used internet from any location during last 30 days','No. of days stayed away from home during last 30 days','No. of meals usually taken in a day','No. of meals taken during last 30 days from school, balwadi etc.','No. of meals taken during last 30 days  others','No. of meals taken during last 30 days from employer as perquisites or part of wage','No. of meals taken during last 30 days at home','No. of meals taken during last 30 days on payment','FDQ original member(generated field)','Status of Member as on revisit','Multiplier']
level_2= pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_02.txt', header=None, colspecs=col_specification, names = col_names)
level_2.head(5)

,Common-ID,Questionnaire No.,Level,Person Srl No.,Relation to head (code),Gender,Age(in years),Marital Status (code),Highest educational level attained (code),Total year of education completed,...,No. of days stayed away from home during last 30 days,No. of meals usually taken in a day,"No. of meals taken during last 30 days from school, balwadi etc.",No. of meals taken during last 30 days others,No. of meals taken during last 30 days from employer as perquisites or part of wage,No. of meals taken during last 30 days at home,No. of meals taken during last 30 days on payment,FDQ original member(generated field),Status of Member as on revisit,Multiplier
0,HCES2022655561010131113011 101202 201,H,2,1,1,1,48,2,6,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,58.0,1,1.0,35560.0
1,HCES2022655561010131113011 101202 201,H,2,2,2,2,46,2,1,NaN,...,0.0,2.0,NaN,NaN,NaN,NaN,60.0,1,1.0,35560.0
2,HCES2022655561010131113011 101202 201,H,2,3,5,1,24,1,13,18.0,...,0.0,2.0,NaN,NaN,NaN,NaN,58.0,1,1.0,35560.0
3,HCES2022655561010131113011 101202 201,H,2,4,5,1,18,1,7,13.0,...,0.0,2.0,NaN,NaN,NaN,NaN,56.0,1,1.0,35560.0
4,HCES2022655561010131113011 101202 201,H,2,5,5,2,21,1,12,17.0,...,0.0,2.0,NaN,NaN,NaN,NaN,54.0,1,1.0,35560.0


In [ ]:
lvl_2 = level_2.value_counts('Common-ID').to_frame().reset_index()
lvl_2.head(5)

,Common-ID,count
0,HCES2022666371080811523175 108112 107,37
1,HCES2022341462363612342036 236101 1312,31
2,HCES2022636211090934223383 109205 303,30
3,HCES2022670621353510123017 235101 203,29
4,HCES2022636811090920723403 109125 308,28


In [ ]:
mult = level_2.groupby('Common-ID')['Multiplier'].unique().to_frame()
mult

,Multiplier
Common-ID,
HCES2022310002282831212025 228111 301,[96498.0]
HCES2022310002282831212025 228111 302,[96498.0]
HCES2022310002282831212025 228111 303,[96498.0]
HCES2022310002282831212025 228111 304,[96498.0]
HCES2022310002282831212025 228111 305,[96498.0]
...,...
HCES2022699991212130123093 221201 312,[125023.0]
HCES2022699991212130123093 221201 313,[125023.0]
HCES2022699991212130123093 221201 314,[125023.0]


In [ ]:
mult_lvl_2 = lvl_2.merge(mult, on ='Common-ID', how='left').fillna(0).reset_index()
mult_lvl_2

,index,Common-ID,count,Multiplier
0,0,HCES2022666371080811523175 108112 107,37,[185297.0]
1,1,HCES2022341462363612342036 236101 1312,31,[185942.0]
2,2,HCES2022636211090934223383 109205 303,30,[208725.0]
3,3,HCES2022670621353510123017 235101 203,29,[8819.0]
4,4,HCES2022636811090920723403 109125 308,28,[139485.0]
...,...,...,...,...
261741,261741,HCES2022699941212130623094 121111 316,1,[188261.0]
261742,261742,HCES2022310022282831012022 228101 306,1,[125146.0]
261743,261743,HCES2022310022282831012022 228101 311,1,[125146.0]
261744,261744,HCES2022310002282831212025 228111 314,1,[96498.0]


In [ ]:
mult_lvl_2['real_count'] = mult_lvl_2['count']*mult_lvl_2['Multiplier']
mult_lvl_2 = mult_lvl_2.drop(['index'],axis=1)
mult_lvl_2

,Common-ID,count,Multiplier,real_count
0,HCES2022666371080811523175 108112 107,37,[185297.0],[6855989.0]
1,HCES2022341462363612342036 236101 1312,31,[185942.0],[5764202.0]
2,HCES2022636211090934223383 109205 303,30,[208725.0],[6261750.0]
3,HCES2022670621353510123017 235101 203,29,[8819.0],[255751.0]
4,HCES2022636811090920723403 109125 308,28,[139485.0],[3905580.0]
...,...,...,...,...
261741,HCES2022699941212130623094 121111 316,1,[188261.0],[188261.0]
261742,HCES2022310022282831012022 228101 306,1,[125146.0],[125146.0]
261743,HCES2022310022282831012022 228101 311,1,[125146.0],[125146.0]
261744,HCES2022310002282831212025 228111 314,1,[96498.0],[96498.0]


In [ ]:
# mergeing data for milk(169)
temp2 = lvl_2.merge(temp1, on='Common-ID', how='left').fillna(0).reset_index()
temp2

,index,Common-ID,count,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
0,0,HCES2022666371080811523175 108112 107,37,F,169.0,2550.0,1852.97,10928.571429
1,1,HCES2022341462363612342036 236101 1312,31,F,169.0,1200.0,1859.42,5142.857143
2,2,HCES2022636211090934223383 109205 303,30,F,169.0,4150.0,2087.25,17785.714286
3,3,HCES2022670621353510123017 235101 203,29,F,169.0,1010.0,88.19,4328.571429
4,4,HCES2022636811090920723403 109125 308,28,F,169.0,3370.0,1394.85,14442.857143
...,...,...,...,...,...,...,...,...
261741,261741,HCES2022699941212130623094 121111 316,1,F,169.0,140.0,1882.61,600.000000
261742,261742,HCES2022310022282831012022 228101 306,1,F,169.0,251.0,1251.46,1075.714286
261743,261743,HCES2022310022282831012022 228101 311,1,F,169.0,244.0,1251.46,1045.714286
261744,261744,HCES2022310002282831212025 228111 314,1,0,0.0,0.0,0.00,0.000000


# Calculating total expanditure on the base of monthly value
level_14['Total Expenditure'] = level_14['Monthly Value']*level_14['Multiplier']
level_14

In [ ]:
FDQ = level_14[level_14['Questionnaire No.'].isin(['F'])]
FDQ.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,90.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,90.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,25.0
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,45.0
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,75.0


FDQ_new = FDQ.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
FDQ_new.reset_index().head(5)

In [ ]:
FDQ_new = FDQ.groupby('Common-ID')['Monthly Value'].sum().to_frame()
FDQ_new.reset_index().head(5)

,Common-ID,Monthly Value
0,HCES2022310002282831212025 228111 301,8704.428571
1,HCES2022310002282831212025 228111 302,13032.428571
2,HCES2022310002282831212025 228111 303,13252.857143
3,HCES2022310002282831212025 228111 304,5644.285714
4,HCES2022310002282831212025 228111 305,1263.142857


In [ ]:
CSQ = level_14[level_14['Questionnaire No.'].isin(['C'])]
CSQ.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
3149267,HCES2022655561010131113011 101202 201,C,349,1960,355.60,1960.0
3149268,HCES2022655561010131113011 101202 301,C,349,1620,303.31,1620.0
3149269,HCES2022655561010131113011 101202 302,C,349,1845,303.31,1845.0
3149270,HCES2022655561010131113011 101202 303,C,349,1520,303.31,1520.0
3149271,HCES2022655561010131113011 101202 304,C,349,1280,303.31,1280.0


CSQ_new = CSQ.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
CSQ_new.reset_index().head(5)

In [ ]:
CSQ_new = CSQ.groupby('Common-ID')['Monthly Value'].sum().to_frame()
CSQ_new.reset_index().head(5)

,Common-ID,Monthly Value
0,HCES2022310002282831212025 228111 301,9655.571429
1,HCES2022310002282831212025 228111 302,24088.469667
2,HCES2022310002282831212025 228111 303,21439.287671
3,HCES2022310002282831212025 228111 304,11276.000000
4,HCES2022310002282831212025 228111 305,9639.000000


In [ ]:
DGQ = level_14[level_14['Questionnaire No.'].isin(['D'])]
DGQ.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value
5689781,HCES2022655561010131113011 101202 201,D,379,8470,355.60,8470.0
5689782,HCES2022655561010131113011 101202 201,D,389,1100,355.60,1100.0
5689783,HCES2022655561010131113011 101202 201,D,399,2210,355.60,2210.0
5689784,HCES2022655561010131113011 101202 301,D,379,7990,303.31,7990.0
5689785,HCES2022655561010131113011 101202 301,D,389,1300,303.31,1300.0


DGQ_new = DGQ.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
DGQ_new.reset_index().head(5)

In [ ]:
DGQ_new = DGQ.groupby('Common-ID')['Monthly Value'].sum().to_frame()
DGQ_new.reset_index().head(5)

,Common-ID,Monthly Value
0,HCES2022310002282831212025 228111 301,16007.000000
1,HCES2022310002282831212025 228111 302,38007.534247
2,HCES2022310002282831212025 228111 303,40848.630137
3,HCES2022310002282831212025 228111 304,20102.479452
4,HCES2022310002282831212025 228111 305,13190.000000


m_fdq = lvl_2.merge(FDQ_new, on='Common-ID', how='left').fillna(0).reset_index()
m_fdq_rnme = m_fdq.rename(columns={'count':'Total_mambers_in_a _household'})
m_fdq_rnme = m_fdq_rnme.rename(columns={'Total Expenditure':'FDQ_Expenditure'})
m_fdq_rnme = m_fdq_rnme.rename(columns={'Common-ID':'Common-ID'})
m_fdq_rnme.head(5)

In [ ]:
m_fdq = lvl_2.merge(FDQ_new, on='Common-ID', how='left').fillna(0).reset_index()
m_fdq_rnme = m_fdq.rename(columns={'count':'Total_mambers_in_a _household'})
m_fdq_rnme = m_fdq_rnme.rename(columns={'Monthly Value':'FDQ_Expenditure'})
m_fdq_rnme = m_fdq_rnme.rename(columns={'Common-ID':'Common-ID'})
m_fdq_rnme.head(5)

,index,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure
0,0,HCES2022666371080811523175 108112 107,37,41464.285714
1,1,HCES2022341462363612342036 236101 1312,31,54941.714286
2,2,HCES2022636211090934223383 109205 303,30,51555.714286
3,3,HCES2022670621353510123017 235101 203,29,67632.857143
4,4,HCES2022636811090920723403 109125 308,28,35376.714286


m_csq = m_fdq_rnme.merge(CSQ_new, on='Common-ID', how='left').fillna(0)
m_csq_rnme = m_csq.rename(columns={'Total Expenditure':'CSQ_Expenditure'})
m_csq_rnme.head(5)

In [ ]:
m_csq = m_fdq_rnme.merge(CSQ_new, on='Common-ID', how='left').fillna(0)
m_csq_rnme = m_csq.rename(columns={'Monthly Value':'CSQ_Expenditure'})
m_csq_rnme.head(5)

,index,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure
0,0,HCES2022666371080811523175 108112 107,37,41464.285714,75531.917808
1,1,HCES2022341462363612342036 236101 1312,31,54941.714286,40329.442270
2,2,HCES2022636211090934223383 109205 303,30,51555.714286,15594.958904
3,3,HCES2022670621353510123017 235101 203,29,67632.857143,42001.722114
4,4,HCES2022636811090920723403 109125 308,28,35376.714286,7943.571429


m_dgq = m_csq_rnme.merge(DGQ_new, on='Common-ID', how='left').fillna(0)
m_dgq_rnme = m_dgq.rename(columns={'Total Expenditure':'DGQ_Expenditure'})
main = m_dgq_rnme.drop(['index'],axis=1)
main.head(5)

In [ ]:
m_dgq = m_csq_rnme.merge(DGQ_new, on='Common-ID', how='left').fillna(0)
m_dgq_rnme = m_dgq.rename(columns={'Monthly Value':'DGQ_Expenditure'})
main = m_dgq_rnme.drop(['index'],axis=1)
main.head(5)

,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure,DGQ_Expenditure
0,HCES2022666371080811523175 108112 107,37,41464.285714,75531.917808,74250.000000
1,HCES2022341462363612342036 236101 1312,31,54941.714286,40329.442270,121801.986301
2,HCES2022636211090934223383 109205 303,30,51555.714286,15594.958904,63739.863014
3,HCES2022670621353510123017 235101 203,29,67632.857143,42001.722114,207784.931507
4,HCES2022636811090920723403 109125 308,28,35376.714286,7943.571429,49614.616438


In [ ]:
main ['one_fdq'] = main['FDQ_Expenditure']/main['Total_mambers_in_a _household']
main ['one_csq'] = main['CSQ_Expenditure']/main['Total_mambers_in_a _household']
main ['one_dgq'] = main['DGQ_Expenditure']/main['Total_mambers_in_a _household']
main.head(5)

,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure,DGQ_Expenditure,one_fdq,one_csq,one_dgq
0,HCES2022666371080811523175 108112 107,37,41464.285714,75531.917808,74250.000000,1120.656371,2041.403184,2006.756757
1,HCES2022341462363612342036 236101 1312,31,54941.714286,40329.442270,121801.986301,1772.313364,1300.949751,3929.096332
2,HCES2022636211090934223383 109205 303,30,51555.714286,15594.958904,63739.863014,1718.523810,519.831963,2124.662100
3,HCES2022670621353510123017 235101 203,29,67632.857143,42001.722114,207784.931507,2332.167488,1448.335245,7164.997638
4,HCES2022636811090920723403 109125 308,28,35376.714286,7943.571429,49614.616438,1263.454082,283.698980,1771.950587


In [ ]:
main['FDQ_Expenditure'].describe()

count    261746.000000
mean       8977.688029
std        5995.689343
min           0.000000
25%        5534.857143
50%        7922.857143
75%       11105.107143
max      757670.428571
Name: FDQ_Expenditure, dtype: float64

In [ ]:
main['CSQ_Expenditure'].describe()

count    2.617460e+05
mean     1.115549e+04
std      1.242663e+04
min      0.000000e+00
25%      5.115401e+03
50%      8.530885e+03
75%      1.387655e+04
max      3.139109e+06
Name: CSQ_Expenditure, dtype: float64

In [ ]:
main['DGQ_Expenditure'].describe()

count    2.617460e+05
mean     2.874372e+04
std      4.902629e+04
min      0.000000e+00
25%      1.168118e+04
50%      1.988510e+04
75%      3.312663e+04
max      3.047822e+06
Name: DGQ_Expenditure, dtype: float64

In [ ]:
x = main['one_fdq'].sum()

In [ ]:
y = main['one_csq'].sum()

In [ ]:
z = main['one_dgq'].sum()
w = x*100/(x+y+z)
w

np.float64(18.207199852930938)

In [ ]:
# Calculating total expanditure on the base of monthly value
mult_applied = level_14.copy()
mult_applied['Total Expenditure'] = mult_applied['Monthly Value']*mult_applied['Multiplier']
mult_applied.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value,Total Expenditure
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,90.0,8001.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,90.0,8001.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,25.0,2222.5
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,45.0,4000.5
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,75.0,105375.0


In [ ]:
FDQ_m = mult_applied[mult_applied['Questionnaire No.'].isin(['F'])]
FDQ_m.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value,Total Expenditure
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,90.0,8001.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,90.0,8001.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,25.0,2222.5
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,45.0,4000.5
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,75.0,105375.0


In [ ]:
FDQ_new_m = FDQ_m.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
FDQ_new_m.reset_index().head(5)

,Common-ID,Total Expenditure
0,HCES2022310002282831212025 228111 301,8.399599e+06
1,HCES2022310002282831212025 228111 302,1.257603e+07
2,HCES2022310002282831212025 228111 303,1.278874e+07
3,HCES2022310002282831212025 228111 304,5.446623e+06
4,HCES2022310002282831212025 228111 305,1.218908e+06


In [ ]:
CSQ_m = mult_applied[mult_applied['Questionnaire No.'].isin(['C'])]
CSQ_m.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value,Total Expenditure
3149267,HCES2022655561010131113011 101202 201,C,349,1960,355.60,1960.0,696976.00
3149268,HCES2022655561010131113011 101202 301,C,349,1620,303.31,1620.0,491362.20
3149269,HCES2022655561010131113011 101202 302,C,349,1845,303.31,1845.0,559606.95
3149270,HCES2022655561010131113011 101202 303,C,349,1520,303.31,1520.0,461031.20
3149271,HCES2022655561010131113011 101202 304,C,349,1280,303.31,1280.0,388236.80


In [ ]:
CSQ_new_m = CSQ_m.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
CSQ_new_m.reset_index().head(5)


,Common-ID,Total Expenditure
0,HCES2022310002282831212025 228111 301,9.317433e+06
1,HCES2022310002282831212025 228111 302,2.324489e+07
2,HCES2022310002282831212025 228111 303,2.068848e+07
3,HCES2022310002282831212025 228111 304,1.088111e+07
4,HCES2022310002282831212025 228111 305,9.301442e+06


In [ ]:
DGQ_m = mult_applied[mult_applied['Questionnaire No.'].isin(['D'])]
DGQ_m.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier,Monthly Value,Total Expenditure
5689781,HCES2022655561010131113011 101202 201,D,379,8470,355.60,8470.0,3011932.0
5689782,HCES2022655561010131113011 101202 201,D,389,1100,355.60,1100.0,391160.0
5689783,HCES2022655561010131113011 101202 201,D,399,2210,355.60,2210.0,785876.0
5689784,HCES2022655561010131113011 101202 301,D,379,7990,303.31,7990.0,2423446.9
5689785,HCES2022655561010131113011 101202 301,D,389,1300,303.31,1300.0,394303.0


In [ ]:
DGQ_new_m = DGQ_m.groupby('Common-ID')['Total Expenditure'].sum().to_frame()
DGQ_new_m.reset_index().head(5)

,Common-ID,Total Expenditure
0,HCES2022310002282831212025 228111 301,1.544643e+07
1,HCES2022310002282831212025 228111 302,3.667651e+07
2,HCES2022310002282831212025 228111 303,3.941811e+07
3,HCES2022310002282831212025 228111 304,1.939849e+07
4,HCES2022310002282831212025 228111 305,1.272809e+07


In [ ]:

m_fdq_m = mult_lvl_2.merge(FDQ_new_m, on='Common-ID', how='left').fillna(0).reset_index()
m_fdq_m_rnme = m_fdq_m.rename(columns={'real_count':'Total_mambers_in_a _household'})
m_fdq_m_rnme = m_fdq_m_rnme.rename(columns={'Total Expenditure':'FDQ_Expenditure'})
m_fdq_m_rnme = m_fdq_m_rnme.rename(columns={'Common-ID':'Common-ID'})
m_fdq_m_rnme.head(5)

,index,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure
0,0,HCES2022666371080811523175 108112 107,37,7.683208e+07
1,1,HCES2022341462363612342036 236101 1312,31,1.021597e+08
2,2,HCES2022636211090934223383 109205 303,30,1.076097e+08
3,3,HCES2022670621353510123017 235101 203,29,5.964542e+06
4,4,HCES2022636811090920723403 109125 308,28,4.934521e+07


In [ ]:
m_csq_m = m_fdq_m_rnme.merge(CSQ_new_m, on='Common-ID', how='left').fillna(0)
m_csq_m_rnme = m_csq_m.rename(columns={'Total Expenditure':'CSQ_Expenditure'})
m_csq_m_rnme.head(5)

,index,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure
0,0,HCES2022666371080811523175 108112 107,37,7.683208e+07,1.399584e+08
1,1,HCES2022341462363612342036 236101 1312,31,1.021597e+08,7.498937e+07
2,2,HCES2022636211090934223383 109205 303,30,1.076097e+08,3.255058e+07
3,3,HCES2022670621353510123017 235101 203,29,5.964542e+06,3.704132e+06
4,4,HCES2022636811090920723403 109125 308,28,4.934521e+07,1.108009e+07


In [ ]:
m_dgq_m = m_csq_m_rnme.merge(DGQ_new_m, on='Common-ID', how='left').fillna(0)
m_dgq_m_rnme = m_dgq_m.rename(columns={'Total Expenditure':'DGQ_Expenditure'})
mult_main = m_dgq_m_rnme.drop(['index'],axis=1)
mult_main.head(5)

,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure,DGQ_Expenditure
0,HCES2022666371080811523175 108112 107,37,7.683208e+07,1.399584e+08,1.375830e+08
1,HCES2022341462363612342036 236101 1312,31,1.021597e+08,7.498937e+07,2.264810e+08
2,HCES2022636211090934223383 109205 303,30,1.076097e+08,3.255058e+07,1.330410e+08
3,HCES2022670621353510123017 235101 203,29,5.964542e+06,3.704132e+06,1.832455e+07
4,HCES2022636811090920723403 109125 308,28,4.934521e+07,1.108009e+07,6.920495e+07


In [ ]:
mult_main ['one_fdq'] = mult_main['FDQ_Expenditure']/mult_main['Total_mambers_in_a _household']
mult_main ['one_csq'] = mult_main['CSQ_Expenditure']/mult_main['Total_mambers_in_a _household']
mult_main ['one_dgq'] = mult_main['DGQ_Expenditure']/mult_main['Total_mambers_in_a _household']
mult_main.head(5)

,Common-ID,Total_mambers_in_a _household,FDQ_Expenditure,CSQ_Expenditure,DGQ_Expenditure,one_fdq,one_csq,one_dgq
0,HCES2022666371080811523175 108112 107,37,7.683208e+07,1.399584e+08,1.375830e+08,2.076543e+06,3.782659e+06,3.718460e+06
1,HCES2022341462363612342036 236101 1312,31,1.021597e+08,7.498937e+07,2.264810e+08,3.295475e+06,2.419012e+06,7.305840e+06
2,HCES2022636211090934223383 109205 303,30,1.076097e+08,3.255058e+07,1.330410e+08,3.586989e+06,1.085019e+06,4.434701e+06
3,HCES2022670621353510123017 235101 203,29,5.964542e+06,3.704132e+06,1.832455e+07,2.056739e+05,1.277287e+05,6.318811e+05
4,HCES2022636811090920723403 109125 308,28,4.934521e+07,1.108009e+07,6.920495e+07,1.762329e+06,3.957175e+05,2.471605e+06


In [ ]:
mult_main['one_fdq'].describe()

count    2.617460e+05
mean     2.432732e+06
std      3.680264e+06
min      0.000000e+00
25%      9.907234e+05
50%      1.878600e+06
75%      2.986955e+06
max      3.404255e+08
Name: one_fdq, dtype: float64

In [ ]:
mult_main['one_csq'].describe()

count    2.617460e+05
mean     3.213979e+06
std      7.795080e+06
min      0.000000e+00
25%      9.138122e+05
50%      1.840346e+06
75%      3.510368e+06
max      1.261644e+09
Name: one_csq, dtype: float64

In [ ]:
mult_main['one_dgq'].describe()

count    2.617460e+05
mean     7.582006e+06
std      2.436279e+07
min      0.000000e+00
25%      2.093358e+06
50%      4.408630e+06
75%      8.116555e+06
max      6.752889e+09
Name: one_dgq, dtype: float64

main['FDQ'] = main['FDQ_Expenditure']/main['Multiplier']
main['CSQ'] = main['CSQ_Expenditure']/main['Multiplier']
main['DGQ'] = main['DGQ_Expenditure']/main['Multiplier']
main = main.drop(['FDQ_Expenditure','CSQ_Expenditure','DGQ_Expenditure'],axis=1)
main.head(5)

st = lvl_2.copy()
st[['part0','part1', 'part2', 'part3']] = st['Common-ID'].apply(split_string)
st

st_copy = st.drop(['Multiplier','part0','part3'],axis=1)
st_copy

st_copy_1 = st_copy.rename(columns={'part1':'Sector'})
st_copy_2= st_copy_1.rename(columns={'part2':'State'})
New_main = main.merge(st_copy_2, on='Common-ID', how='left').fillna(0)
New_main

check = New_main[New_main['Sector'].isin(['1'])]
check.head(5)

x = New_main['State']
y = New_main['FDQ']
z = New_main['DGQ']
w = New_main['CSQ']

# Create the scatter plot
plt.scatter(x, y)

# Add title and labels
plt.title("Scatter Plot Example")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")

# Show the plot
plt.show()

# Create the scatter plot
plt.scatter(x, z)

# Add title and labels
plt.title("Scatter Plot Example")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")

# Show the plot
plt.show()

# Create the scatter plot
plt.scatter(x, w)

# Add title and labels
plt.title("Scatter Plot Example")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")

# Show the plot
plt.show()